# Problem 7: Word Vectors - Manual NN (20 Newsgroups)

**Dataset:** 20 Newsgroups (5-8 categories)  
**Method:** Neural network trained on PPMI matrix (Manual NumPy)

In [1]:
import numpy as np
from collections import Counter
import re
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups

## 1. Preprocessing

In [2]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z ]+", ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.split()

def build_vocabulary(words, vocab_size=5000, min_count=3):
    stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'from',
                  'is', 'was', 'were', 'be', 'been', 'have', 'has', 'had', 'this', 'that', 'it', 'as',
                  'will', 'would', 'could', 'can', 'do', 'does', 'did', 'not', 'no', 'if', 'so'}
    
    word_counts = Counter(words)
    filtered = [(w, c) for w, c in word_counts.items() 
                if c >= min_count and len(w) >= 3 and w not in stop_words]
    
    most_common = sorted(filtered, key=lambda x: x[1], reverse=True)[:vocab_size]
    word_to_id = {word: idx for idx, (word, _) in enumerate(most_common)}
    id_to_word = {idx: word for word, idx in word_to_id.items()}
    corpus = [word_to_id[word] for word in words if word in word_to_id]
    
    return word_to_id, id_to_word, corpus

## 2. Co-occurrence Matrix

In [3]:
def build_cooccurrence_matrix(corpus, vocab_size, window_size=5):
    cooccur = np.zeros((vocab_size, vocab_size), dtype=np.float32)
    
    for i in tqdm(range(len(corpus)), desc="Building co-occurrence"):
        center = corpus[i]
        start = max(0, i - window_size)
        end = min(len(corpus), i + window_size + 1)
        
        for j in range(start, end):
            if i != j:
                context = corpus[j]
                distance = abs(i - j)
                weight = 1.0 / distance
                cooccur[center, context] += weight
    
    return cooccur

## 3. PPMI Computation

In [4]:
def compute_ppmi(cooccur_matrix):
    total = cooccur_matrix.sum()
    word_counts = cooccur_matrix.sum(axis=1)
    context_counts = cooccur_matrix.sum(axis=0)
    
    ppmi = np.zeros_like(cooccur_matrix)
    
    for i in tqdm(range(cooccur_matrix.shape[0]), desc="Computing PPMI"):
        for j in range(cooccur_matrix.shape[1]):
            if cooccur_matrix[i, j] > 0:
                p_ij = cooccur_matrix[i, j] / total
                p_i = word_counts[i] / total
                p_j = context_counts[j] / total
                
                pmi = np.log(p_ij / (p_i * p_j + 1e-10))
                ppmi[i, j] = max(0, pmi)
    
    np.fill_diagonal(ppmi, 0)
    row_sums = ppmi.sum(axis=1, keepdims=True)
    ppmi_normalized = np.divide(ppmi, row_sums, where=row_sums>0)
    
    return ppmi_normalized

## 4. Neural Network

In [5]:
class WordVectorNN:
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
        self.W1 = np.random.normal(0, 0.1, (vocab_size, embedding_dim)).astype(np.float32)
        self.W2 = np.random.normal(0, 0.1, (embedding_dim, vocab_size)).astype(np.float32)
    
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / (np.sum(exp_x) + 1e-10)
    
    def forward(self, word_idx):
        hidden = self.W1[word_idx]
        output = hidden @ self.W2
        probs = self.softmax(output)
        return hidden, probs
    
    def backward(self, word_idx, hidden, probs, target, lr):
        d_output = probs - target
        d_W2 = np.outer(hidden, d_output)
        d_hidden = d_output @ self.W2.T
        
        self.W2 -= lr * d_W2
        self.W1[word_idx] -= lr * d_hidden
    
    def train_step(self, word_idx, target, lr):
        hidden, probs = self.forward(word_idx)
        loss = -np.sum(target * np.log(probs + 1e-10))
        self.backward(word_idx, hidden, probs, target, lr)
        return loss
    
    def get_embedding(self, word_idx):
        return self.W1[word_idx]

## 5. Training

In [6]:
def train_model(model, ppmi_matrix, epochs=10, lr=0.789):
    vocab_size = ppmi_matrix.shape[0]
    
    for epoch in range(epochs):
        total_loss = 0
        indices = np.arange(vocab_size)
        np.random.shuffle(indices)
        
        for idx in tqdm(indices, desc=f"Epoch {epoch+1}/{epochs}"):
            loss = model.train_step(idx, ppmi_matrix[idx], lr)
            total_loss += loss
        
        avg_loss = total_loss / vocab_size
        print(f"Epoch {epoch+1}/{epochs}, Avg Loss: {avg_loss:.4f}")

## 6. Evaluation

In [7]:
def cosine_similarity(v1, v2):
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    if norm1 == 0 or norm2 == 0:
        return 0
    return np.dot(v1, v2) / (norm1 * norm2)

def find_similar_words(word, word_to_id, id_to_word, model, top_k=15):
    if word not in word_to_id:
        return None
    
    word_idx = word_to_id[word]
    word_emb = model.get_embedding(word_idx)
    
    similarities = []
    for idx in range(model.vocab_size):
        if idx != word_idx:
            other_emb = model.get_embedding(idx)
            sim = cosine_similarity(word_emb, other_emb)
            similarities.append((id_to_word[idx], sim))
    
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

def evaluate_model(model, test_words, word_to_id, id_to_word):
    for word in test_words:
        similar = find_similar_words(word, word_to_id, id_to_word, model)
        if similar:
            print(f"\n{word.upper()}:")
            for w, sim in similar:
                print(f"  {w:20s} {sim:.4f}")

## 7. Run Pipeline

In [8]:
categories = [
    'comp.graphics', 'comp.sys.ibm.pc.hardware',
    'sci.crypt', 'sci.space',
    'soc.religion.christian',
    'talk.politics.guns', 'talk.politics.mideast',
    'alt.atheism'
]
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))

text = ' '.join(newsgroups.data)
words = preprocess_text(text)
print(f"Total words: {len(words):,}")

print("\nBuilding vocabulary...")
word_to_id, id_to_word, corpus = build_vocabulary(words, vocab_size=10000)
print(f"Vocab: {len(word_to_id):,}, Corpus: {len(corpus):,}")

print("\nBuilding co-occurrence matrix...")
cooccur = build_cooccurrence_matrix(corpus, len(word_to_id), window_size=5)

print("\nComputing PPMI...")
ppmi = compute_ppmi(cooccur)

print("\nInitializing model...")
model = WordVectorNN(vocab_size=len(word_to_id), embedding_dim=200)

print("\nTraining...")
train_model(model, ppmi, epochs=10, lr=0.789)

print("\n" + "="*60)
print("EVALUATION")
print("="*60)
test_words = ["china", "computer", "phone", "napoleon", "god", "catholic"]
evaluate_model(model, test_words, word_to_id, id_to_word)

Loading 20 Newsgroups...
Total words: 1,033,364

Building vocabulary...
Vocab: 10,000, Corpus: 569,458

Building co-occurrence matrix...


Building co-occurrence: 100%|██████████| 569458/569458 [00:01<00:00, 493245.05it/s]



Computing PPMI...


Computing PPMI: 100%|██████████| 10000/10000 [00:07<00:00, 1266.09it/s]



Initializing model...

Training...


...

Epoch 8/10, Avg Loss: 8.3245


Epoch 9/10: 100%|██████████| 10000/10000 [00:14<00:00, 688.37it/s]


Epoch 9/10, Avg Loss: 8.1439


Epoch 10/10: 100%|██████████| 10000/10000 [00:13<00:00, 729.20it/s]


Epoch 10/10, Avg Loss: 7.9555

EVALUATION

CHINA:
  hong                 0.3731
  kong                 0.3641
  scheduled            0.3578
  sponsor              0.3304
  korean               0.3264
  cosmos               0.3249
  worldview            0.3243
  ariane               0.3230
  boycott              0.3157
  titan                0.3146
  kansas               0.3144
  usaf                 0.3118
  saturn               0.3102
  chinese              0.3088
  distributor          0.3079

COMPUTER:
  applications         0.3993
  ncsa                 0.3953
  advanced             0.3848
  verlag               0.3766
  ieee                 0.3740
  tel                  0.3684
  pub                  0.3635
  corp                 0.3608
  interactive          0.3588
  xerox                0.3564
  geometry             0.3561
  convex               0.3536
  systems              0.3529
  laboratory           0.3523
  idl                  0.3490

PHONE:
  cost                 0.3811
 

The same idea as Text8 but on newsgroup discussions. The network learns word meaning from how people use them in different discussion topics like tech, politics, sports. By training on multiple categories, the model learns that words like "ball" mean different things in sports vs political contexts. This helps nn know why context matters and the network figures out word relationships by seeing how they are actually used in real conversations. 

By training on 8 different newsgroup categories, the model sees words used in various contexts. "windows" appears near "microsoft", "software" in tech groups but "windows" might appear near  "house", "glass", "frame" in home groups, so context matters and model learning both meanings from context.

similar algorithm:
1. PPMI
2. co-occurrence matrix with window_size 5
3. inverse distace weighting
4. Neural network: vocab_size * embedding_dim * vocab_size
5. manual backpropagation